# HIMATEKTRO ITS SOCIAL MEDIA ANALYTICS

### Key Steps:
1. Data Crawling/Scrapping \
https://medium.com/@Arifku__/scraping-komentar-instagram-menggunakan-selenium-python-3de175b3249d 
https://stackoverflow.com/questions/61069660/how-to-scrape-instagram-using-python-using-selenium-after-instagram-changed-thei
https://kiwidamien.github.io/webscraping-beyond-beautifulsoup-and-selenium.html
2. Data Cleanup
3. Explolatory Analysis
4. Clustering Post

## 1. Data Crawling

In [1]:
from bs4 import BeautifulSoup as bs 
import selenium as sl
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from pandas import ExcelWriter
import os.path

ModuleNotFoundError: No module named 'bs4'

In [2]:
driver = webdriver.Chrome(executable_path=r'E:\donlot\chromedriver_win32\chromedriver')

In [3]:
driver.get('https://www.instagram.com/accounts/login/')
usernameInput='YOURUSER'
passwordInput='YOURPASSWORD'
username=WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='username']"))) .send_keys(usernameInput)
password=WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='password']"))).send_keys(passwordInput)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='password']"))).send_keys(Keys.ENTER)
time.sleep(15)

In [4]:
driver.get('https://www.instagram.com/himatektro_its/')

#element = driver.find_element_by_tag_name('body') # or whatever tag you're looking to scrape from

#for i in range(15):
#    element.send_keys(Keys.END)
#    time.sleep(1)
#    source = driver.page_source
    
source = driver.page_source
soup=bs(source, 'html.parser')

body = soup.find('body')
script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
page_json = script.string.split(' = ', 1)[1].rstrip(';')
data = json.loads(page_json)

In [5]:
links = []
for each in data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']:
    link = 'https://www.instagram.com'+'/p/'+each['node']['shortcode']+'/'
    posttext = each['node']['edge_media_to_caption']['edges'][0]['node']['text'].replace('\n','')
    comments = each['node']['edge_media_to_comment']['count']
    likes = each['node']['edge_liked_by']['count']
    postimage = each['node']['thumbnail_src']
    isvideo = each['node']['is_video']
    postdate = time.strftime('%Y %b %d %H:%M:%S', time.localtime(each['node']['taken_at_timestamp']))
    links.append([link, posttext, comments, likes, postimage, isvideo, postdate])

In [6]:
table = pd.DataFrame(links)
table.columns = ['Link', 'Caption', 'Comments', 'Likes', 'Image Link', 'Video?','Post Date']
table

,Link,Caption,Comments,Likes,Image Link,Video?,Post Date
0,https://www.instagram.com/p/CIdsW_MjMe4/,[PENDAFTARAN LFF HIMATEKTRO ITS 2020]Halo HIMA...,0,55,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 07 00:02:13
1,https://www.instagram.com/p/CIamEeIjRIh/,[KATA MEREKA TENTANG LFF]LFF adalah sebuah pel...,0,112,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 05 19:09:32
2,https://www.instagram.com/p/CIX-6TEDbMh/,[EXTENDED SAYEMBARA KWU]Halo Himatektro ITS!!!...,0,44,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 04 18:48:53
3,https://www.instagram.com/p/CITFN_1DEZ9/,[COMING SOON: LFF?]Yang ditunggu-tunggu telah ...,0,57,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 02 21:07:48
4,https://www.instagram.com/p/CIS-Q3QjJqE/,[Apresiasi Kader] Part 8Setiap orang berhak me...,0,110,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 02 20:07:01
5,https://www.instagram.com/p/CIS5DTTD6JJ/,"[Manusia akan Mati, dan Dilupakan. Tapi Karya ...",0,71,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 02 19:21:29
6,https://www.instagram.com/p/CISyMD3jvNB/,[INTIP.HIMA: DESEMBER 2020]Halo HIMATEKTRO ITS...,0,78,https://scontent-xsp1-1.cdninstagram.com/v/t51...,True,2020 Dec 02 18:22:11
7,https://www.instagram.com/p/CIShOj0jmwP/,[INFORMASI BATAS DROP MATA KULIAH]Hallo HIMATE...,0,54,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 02 15:53:18
8,https://www.instagram.com/p/CIQPEljDJH3/,[PKTI-TD 2020]PERSIAPKAN! Kembalikan Kejayaan!...,0,103,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 01 18:36:10
9,https://www.instagram.com/p/CIPhP2Lj3t3/,[INFOPRESMA]Haloo sobat Himatektro...Udah bula...,6,233,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 01 11:55:45


### Query More Post
The above method surprisingly only giving us 12 post at max. We need more.. so lets try another method

In [7]:
import requests

#### Use the end-cursor recursive to get into maximum post

In [97]:
#Autoget profile ID
profilename = 'himatektro_its'
instagramlink = 'https://www.instagram.com/'+profilename+'/?__a=1'
getprofile = requests.get(instagramlink)
idjson = profilq.json()
idnumber = idjson['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node']['owner']['id']

#combine mainlink. initiate recursive
mainlink = 'https://www.instagram.com/graphql/query/?query_hash=42323d64886122307be10013ad2dcc44&variables={%22id%22:%22'+idnumber+'%22,%22first%22:%2250%22,%22after%22:%22'
endcursor = ''
linksq=[]
cursor = []
for i in range(25):
    try:
        print('getting... ',i)
        iteration = i
        combinelink = mainlink + endcursor + '%22}'
        profilq = requests.get(combinelink)
        j = profilq.json()
        for each in j['data']['user']['edge_owner_to_timeline_media']['edges']:
            try:
                link = 'https://www.instagram.com'+'/p/'+each['node']['shortcode']+'/'
                posttext = each['node']['edge_media_to_caption']['edges'][0]['node']['text'].replace('\n','') 
                comments = each['node']['edge_media_to_comment']['count']
                likes = each['node']['edge_media_preview_like']['count']
                thumbnail = each['node']['thumbnail_src']
                isvideo = each['node']['is_video']
                postdate = time.strftime('%Y %b %d %H:%M:%S', time.localtime(each['node']['taken_at_timestamp']))
            #this is to avoid error when there's blank in caption (case based on HIMATEKTRO ITS)
            except IndexError:
                posttext = "**no caption provided**"
            linksq.append([link, posttext, comments, likes, thumbnail, isvideo, postdate])
        tableq = pd.DataFrame(linksq)
        cursor.append([iteration, endcursor])
        endcursor = j['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
    #this is to avoid recursive error in the last page 
    except TypeError:
        pass
    except KeyError:
        pass
tableq.columns = ['Link', 'Caption', 'Comments', 'Likes', 'Thumbnail', 'Video?','Post Date']
tablecursor = pd.DataFrame(cursor)
tablecursor.columns = ['iteration', 'endcursor hash']

getting...  0
getting...  1
getting...  2
getting...  3
getting...  4
getting...  5
getting...  6
getting...  7
getting...  8
getting...  9
getting...  10
getting...  11
getting...  12
getting...  13
getting...  14
getting...  15
getting...  16
getting...  17
getting...  18
getting...  19
getting...  20
getting...  21
getting...  22
getting...  23
getting...  24


In [98]:
tableq

,Link,Caption,Comments,Likes,Thumbnail,Video?,Post Date
0,https://www.instagram.com/p/CIesTXBjeOK/,[SAFARI DOSEN]Wabah Covid-19 membuat kita haru...,0,44,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 07 09:20:58
1,https://www.instagram.com/p/CIdsW_MjMe4/,[PENDAFTARAN LFF HIMATEKTRO ITS 2020]Halo HIMA...,0,63,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 07 00:02:13
2,https://www.instagram.com/p/CIamEeIjRIh/,[KATA MEREKA TENTANG LFF]LFF adalah sebuah pel...,0,113,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 05 19:09:32
3,https://www.instagram.com/p/CIX-6TEDbMh/,[EXTENDED SAYEMBARA KWU]Halo Himatektro ITS!!!...,0,44,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 04 18:48:53
4,https://www.instagram.com/p/CITFN_1DEZ9/,[COMING SOON: LFF?]Yang ditunggu-tunggu telah ...,0,57,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Dec 02 21:07:48
...,...,...,...,...,...,...,...
1159,https://www.instagram.com/p/BMk0bqMglU1/,**no caption provided**,0,29,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2016 Nov 21 08:33:14
1160,https://www.instagram.com/p/BMk0axrgahI/,**no caption provided**,0,29,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2016 Nov 21 08:33:14
1161,https://www.instagram.com/p/BMk0ZvdgpXH/,**no caption provided**,0,29,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2016 Nov 21 08:33:14
1162,https://www.instagram.com/p/BMk0Y3ggw9W/,**no caption provided**,0,29,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2016 Nov 21 08:33:14


In [75]:
tableq.count()

Link         1164
Caption      1164
Comments     1164
Likes        1164
Thumbnail    1164
Video?       1164
Post Date    1164
dtype: int64

In [82]:
tableq.to_csv(r'D:\Vito\T&T\Data Science\Praktek\scrapping.csv')

In [69]:
tablecursor

,iteration,endcursor hash
0,0,
1,1,QVFERjZrTjVnR09YSWk3Y3ozYmNTY3kzYzVuZUVsQnh3LT...
2,2,QVFDT1NEekN3THFLS0tHcFhiVUtIMjBmQURoYmxwQzZORD...
3,3,QVFDNTBRWkswc18yT3picG5XamZaVWpSNVppUVJ6WGtLNl...
4,4,QVFDQnRVWVlSNGpyUWJweW56a2pUY2FsVG9OWUx2b1c0R1...
5,5,QVFCR2llZWk3VXZHWjdiWF9PUkQzUkJyeFVMQVJPUVphTW...
6,6,QVFEWHRrUzdVYWFaM3czUzBNd01ieThHWUxWM05XR1AzTE...
7,7,QVFETkJDM3V0dm51a25uRUxPNmtKUDFlV1JKR2VVQVVlMz...
8,8,QVFBNmVuLVQ4bTlTRXQtWURkRVdNRnBxUzc0MDV0ZHl5aV...
9,9,QVFEUGlvcTlodHVEcVNtdDZCRXhNNFFOQmQ0d2NYcGh2OW...


## 3. Exploratory Analysis

In [85]:
tableq.describe()

,Comments,Likes
count,1164.000000,1164.000000
mean,0.908935,64.806701
std,3.943752,38.032858
min,0.000000,8.000000
25%,0.000000,41.000000
50%,0.000000,54.000000
75%,0.000000,75.250000
max,90.000000,358.000000


From the describe function, we can conclude that HIMATEKTRO ITS have an average comment less than 1. With average likes around 64 

In [83]:
tableq.nlargest(10, columns='Likes').tail(10)

,Link,Caption,Comments,Likes,Thumbnail,Video?,Post Date
89,https://www.instagram.com/p/CF6F2DKDGJ2/,"Haii, udah semester baru aja nih! Udah tau bel...",10,358,https://scontent-xsp1-1.cdninstagram.com/v/t51...,True,2020 Oct 04 11:20:47
117,https://www.instagram.com/p/CFEg5-6Duj1/,[RANGKING IPS SEMESTER GENAP]Salam Prestasi!Se...,19,300,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Sep 13 15:46:56
301,https://www.instagram.com/p/B9v9dzRFP3k/,[DINAKARA PAMIT UNDUR DIRI]Berdasarkan 14/TAP/...,5,290,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Mar 15 16:32:41
336,https://www.instagram.com/p/B7Hn-8ClAjQ/,"[TERIMAKASIH DAN SELAMAT BERTUGAS]Dekade baru,...",0,277,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Jan 10 07:32:34
304,https://www.instagram.com/p/B9f2aLtFu09/,[ RANKING IPS SEMESTER GASAL ]Salam prestasi!S...,15,268,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2020 Mar 09 10:23:11
68,https://www.instagram.com/p/CGkK1Bpjnjm/,[RETRO]Halo Adik-adik Calon Mahasiswa Baru! Te...,5,253,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Oct 20 19:21:07
423,https://www.instagram.com/p/B3JASNKl1Qt/,[RANKING IPS SEMESTER GENAP]Salam prestasi!Sel...,12,250,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2019 Oct 03 10:18:34
406,https://www.instagram.com/p/B3rik6xlGde/,[STAFF OF THE MONTH]Safar membuka kaleng teras...,50,240,https://scontent-xsp1-1.cdninstagram.com/v/t51...,False,2019 Oct 16 20:12:24
214,https://www.instagram.com/p/CB-RMWnjxhm/,[ BERITA DUKA ]Turut berduka cita atas wafatny...,1,236,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Jun 28 16:00:01
10,https://www.instagram.com/p/CIPhP2Lj3t3/,[INFOPRESMA]Haloo sobat Himatektro...Udah bula...,6,233,https://scontent-xsp1-2.cdninstagram.com/v/t51...,False,2020 Dec 01 11:55:45


The 10 most liked posts of all time are those who comes from 2019-2020. This is might be due to more follower at present time. So this is not valid aggregation

In [99]:
mainlink

'https://www.instagram.com/graphql/query/?query_hash=42323d64886122307be10013ad2dcc44&variables={%22id%22:%223970527928%22,%22first%22:%2250%22,%22after%22:%22'